# CV intro

In [ ]:
! pip install torch==1.8.0 "catalyst[cv]"==21.03 "catalyst[ml]"==21.03

In [ ]:
import numpy as np
import random
import torch

In [ ]:
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

## Numpy and Pytorch intro
### Initialization

In [ ]:
a = [1. , 1.4 , 2.5]
print(f"Simple way: {torch.tensor(a)}")
print(f"Zeros:\n {torch.zeros((2,3))}")
print(f"Range: {torch.arange(0, 10)}")
print(f"Complicated range: {torch.arange(4, 12, 2)}")
print(f"Space: {torch.linspace(1, 4, 6)}")
print(f"Identity matrix:\n {torch.eye(4)}")

### Random

In [ ]:
print(f"From 0 to 1: {torch.rand(1)}")
print(f"Vector from 0 to 1: {torch.rand(5)}")
print(f"Vector from 0 to 10: {torch.randint(10, size=(5,))}")

### Matrix Operation

In [ ]:
a = torch.arange(10).type(torch.FloatTensor)
b = torch.linspace(-10, 10, 10)
print(f"a: {a}\nshape: {a.size()}")
print(f"b: {a}\nshape: {b.size()}")
print(f"a + b: {a + b},\n a * b: {a * b}")
print(f"Dot product: {a.dot(b)}")
print(f"Mean: {a.mean()}, STD: {a.std()}")
print(f"Sum: {a.sum()}, Min: {a.min()}, Max: {a.max()}")
print(f"Reshape:\n{a.reshape(-1, 1)}\nshape: {a.reshape(-1, 1).size()}")
c = a.reshape(-1, 1).repeat(1, 5)
print(f"Repeat:\n{c}\nshape: {c.size()}")
print(f"Transpose:\n{c.T}\nshape: {c.T.size()}")
print(f"Unique items: {torch.unique(c)}")

### Indexing

In [ ]:
a = torch.arange(100).reshape(10, 10)
print(f"Array:\n{a}\nshape: {a.size()}")
print(f"Get first column: {a[:, 0]}")
print(f"Get last row: {a[-1, :]}")
print(f"Add new awis:\n{a[:, np.newaxis]}\nshape: {a[:, np.newaxis].size()}")
print(f"Specific indexing:\n{a[4:6, 7:]}")

### Numpy <-> Pytorch

In [ ]:
a = torch.normal(mean=torch.zeros(2,4))
a.numpy()

In [ ]:
b = np.random.normal(size=(2, 4))
torch.from_numpy(b)

### CUDA

In [ ]:
a = torch.normal(mean=torch.zeros(2,4))
b = torch.normal(mean=torch.zeros(2, 4))
print(f"a:\n{a}\nb:\n{b}")

In [ ]:
a = a.cuda()

In [ ]:
a + b

In [ ]:
(a + b.cuda()).cpu()

### Autograd

In [ ]:
a = torch.randn(2, requires_grad=True)
b = torch.normal(mean=torch.zeros(2))

c = torch.dot(a, b)
print(f'a:\n{a}\nb:\n{b}\n(a,b): {c}')

In [ ]:
c.backward()
print(f'a:\n{a}\nb:\n{b}\n(a,b): {c}')
print(f"Grad a: {a.grad}")

In [ ]:
a = torch.randn(2, requires_grad=True)
b = torch.normal(mean=torch.zeros(2))
c = torch.ones(1, requires_grad=True)

d = torch.sigmoid(torch.dot(a, b) + c)
print(f'a:\n{a}\nb:\n{b}\nSigmoid( (a,b) ): {d}')

print(f"Grad a: {a.grad}\nGrad c: {c.grad}")

In [ ]:
d.backward()
print(f"Grad a: {a.grad}\nGrad c: {c.grad}")

---

# PyTorch 101

In [ ]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from catalyst import dl
from catalyst.data.transforms import ToTensor
from catalyst.contrib.datasets import MNIST

### MNIST 101

In [ ]:
train_loader = DataLoader(
    MNIST(os.getcwd(), train=True, download=True, transform=ToTensor()),
    batch_size=32,
)
valid_loader = DataLoader(
    MNIST(os.getcwd(), train=False, download=True, transform=ToTensor()),
    batch_size=32,
)

model = nn.Sequential(nn.Flatten(), nn.Linear(28 * 28, 10)).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [ ]:
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import seaborn as sns
sns.set()

%matplotlib inline

def train(model, optimizer, n_epoch=1, device="cpu"):
    train_logs = {"Train Loss": [0,], "Steps": [0,]}
    valid_logs = {"Valid Loss": [0,], "Valid Accuracy": [0,], "Steps": [0,]}
    step = 0
    best_valid_loss = np.inf
    best_model = None

    for i in range(n_epoch):
        # train
        for x_batch, y_batch in train_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            
            predictions = model(x_batch)
            loss = criterion(predictions, y_batch)

            loss.backward()
            
            optimizer.step()   
            
            step += 1
            train_logs["Train Loss"].append(loss.detach().item())
            train_logs["Steps"].append(step)

        # valid
        sum_loss = 0
        sum_acc = 0
        count_valid_steps = 0
        with torch.no_grad():
            for x_batch, y_batch in valid_loader:
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                predictions = model(x_batch)
                loss = criterion(predictions, y_batch)
                sum_loss += loss.item()
                sum_acc += accuracy_score(y_batch.cpu().numpy(), np.argmax(predictions.cpu().numpy(), axis=1))
                count_valid_steps += 1

            valid_logs["Valid Loss"].append(sum_loss / count_valid_steps)
            valid_logs["Valid Accuracy"].append(sum_acc / count_valid_steps)
            valid_logs["Steps"].append(step)

            if best_valid_loss > sum_loss / count_valid_steps:
                best_valid_loss = sum_loss / count_valid_steps
                best_model = deepcopy(model)

    fig, ax = plt.subplots(1, 3, figsize=(20, 5))
    sns.lineplot(x="Steps", y="Train Loss", data=train_logs, ax=ax[0])
    sns.lineplot(x="Steps", y="Valid Loss", data=valid_logs, ax=ax[1])
    sns.lineplot(x="Steps", y="Valid Accuracy", data=valid_logs, ax=ax[2])
    plt.plot()

    return best_model, train_logs, valid_logs

In [ ]:
model, _, _ = train(model, optimizer, n_epoch=2, device="cuda")

In [ ]:
# PyTorch is great, but...
# sometimes it's too lowlevel
# https://github.com/NVlabs/stylegan2-ada-pytorch/blob/main/training/training_loop.py#L88
# https://github.com/NVlabs/imaginaire/blob/master/imaginaire/trainers/base.py#L24

# [Catalyst](https://github.com/catalyst-team/catalyst) 101
PyTorch framework for Deep Learning research and development. It focuses on reproducibility, rapid experimentation, and codebase reuse so you can create something new rather than write another regular train loop.

Break the cycle - use the Catalyst!

tl;dr
- [minimal examples](https://github.com/catalyst-team/catalyst#minimal-examples)
- [docs](https://catalyst-team.github.io/catalyst/)
- [Config API](https://github.com/catalyst-team/catalyst/tree/master/examples/mnist_stages) (advanced)

In [ ]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from catalyst import dl
from catalyst.data.transforms import ToTensor
from catalyst.contrib.datasets import MNIST

In [ ]:
train_loader = DataLoader(
    MNIST(os.getcwd(), train=True, download=True, transform=ToTensor()),
    batch_size=32,
)
valid_loader = DataLoader(
    MNIST(os.getcwd(), train=False, download=True, transform=ToTensor()),
    batch_size=32,
)
loaders = {"train": train_loader, "valid": valid_loader}

model = nn.Sequential(nn.Flatten(), nn.Linear(28 * 28, 10))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

### MNIST 101

In [ ]:
runner = dl.SupervisedRunner(input_key="features", output_key="logits", target_key="targets", loss_key="loss")
# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    num_epochs=5,
    callbacks=[
        dl.AccuracyCallback(input_key="logits", target_key="targets", topk_args=(1, 3, 5)),
    ],
    logdir="./logs/mnist_101",
    valid_loader="valid",
    valid_metric="loss",
    minimize_valid_metric=True,
    verbose=True,
    load_best_on_end=True,
)

### MNIST 102

In [ ]:
runner = dl.SupervisedRunner(input_key="features", output_key="logits", target_key="targets", loss_key="loss")
# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    num_epochs=5,
    logdir="./logs/mnist_102",
    callbacks=[
        dl.AccuracyCallback(input_key="logits", target_key="targets", num_classes=10, log_on_batch=True),
        dl.PrecisionRecallF1SupportCallback(input_key="logits", target_key="targets", num_classes=10, log_on_batch=True),
        dl.AUCCallback(input_key="logits", target_key="targets"),
        # catalyst[ml] required
        dl.ConfusionMatrixCallback(input_key="logits", target_key="targets", num_classes=10),
        dl.CheckpointCallback(
            logdir="./logs/mnist_102", 
            loader_key="valid", 
            metric_key="accuracy01", 
            minimize=False, 
            save_n_best=3, 
            load_on_stage_end="best"
        ),
        dl.TqdmCallback()
    ],
)

In [ ]:
# complicated console output?
# time for an advanced logging!

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

---

# Catalyst 102

### MNIST metric learning
- [Metric Learning with Catalyst](https://medium.com/pytorch/metric-learning-with-catalyst-8c8337dfab1a)
- [Representation Learning with Catalyst and Faces](https://medium.com/catalyst-team/representation-learning-with-catalyst-and-faces-946644d49184)

In [ ]:
import warnings
warnings.simplefilter("ignore")
import os
from torch.optim import Adam
from torch.utils.data import DataLoader
from catalyst import data, dl
from catalyst.contrib import datasets, models, nn
from catalyst.data.transforms import Compose, Normalize, ToTensor


# 1. train and valid loaders
transforms = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])

train_dataset = datasets.MnistMLDataset(root=os.getcwd(), download=True, transform=transforms)
sampler = data.BalanceBatchSampler(labels=train_dataset.get_labels(), p=5, k=10)
train_loader = DataLoader(dataset=train_dataset, sampler=sampler, batch_size=sampler.batch_size)

valid_dataset = datasets.MnistQGDataset(root=os.getcwd(), transform=transforms, gallery_fraq=0.2)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=1024)

# 2. model and optimizer
model = models.MnistSimpleNet(out_features=16)
optimizer = Adam(model.parameters(), lr=0.001)

# 3. criterion with triplets sampling
sampler_inbatch = data.HardTripletsSampler(norm_required=False)
criterion = nn.TripletMarginLossWithSampler(margin=0.5, sampler_inbatch=sampler_inbatch)

# 4. training with catalyst Runner
class CustomRunner(dl.SupervisedRunner):
    def handle_batch(self, batch) -> None:
        if self.is_train_loader:
            images, targets = batch["features"].float(), batch["targets"].long()
            features = self.model(images)
            self.batch = {"embeddings": features, "targets": targets,}
        else:
            images, targets, is_query = batch["features"].float(), batch["targets"].long(), batch["is_query"].bool()
            features = self.model(images)
            self.batch = {"embeddings": features, "targets": targets, "is_query": is_query}

callbacks = [
    dl.ControlFlowCallback(
        dl.CriterionCallback(
            input_key="embeddings", target_key="targets", metric_key="loss"
        ),
        loaders="train",
    ),
    dl.ControlFlowCallback(
        dl.CMCScoreCallback(
            embeddings_key="embeddings",
            labels_key="targets",
            is_query_key="is_query",
            topk_args=[1],
        ),
        loaders="valid",
    ),
    dl.PeriodicLoaderCallback(
        valid_loader_key="valid", valid_metric_key="cmc01", minimize=False, valid=2
    ),
    dl.CheckpointCallback(logdir="./logs/ml", loader_key="valid", metric_key="cmc01", minimize=False)
]

runner = CustomRunner(input_key="features", output_key="embeddings")
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    callbacks=callbacks,
    loaders={"train": train_loader, "valid": valid_loader},
    verbose=False,
    logdir="./logs/ml",
#     valid_loader="valid",
#     valid_metric="cmc01",
#     minimize_valid_metric=False,
    num_epochs=10,
)

### MNIST GAN

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from catalyst import dl
from catalyst.contrib.datasets import MNIST
from catalyst.contrib.nn.modules import Flatten, GlobalMaxPool2d, Lambda
from catalyst.data.transforms import ToTensor

latent_dim = 128
generator = nn.Sequential(
    # We want to generate 128 coefficients to reshape into a 7x7x128 map
    nn.Linear(128, 128 * 7 * 7),
    nn.LeakyReLU(0.2, inplace=True),
    Lambda(lambda x: x.view(x.size(0), 128, 7, 7)),
    nn.ConvTranspose2d(128, 128, (4, 4), stride=(2, 2), padding=1),
    nn.LeakyReLU(0.2, inplace=True),
    nn.ConvTranspose2d(128, 128, (4, 4), stride=(2, 2), padding=1),
    nn.LeakyReLU(0.2, inplace=True),
    nn.Conv2d(128, 1, (7, 7), padding=3),
    nn.Sigmoid(),
)
discriminator = nn.Sequential(
    nn.Conv2d(1, 64, (3, 3), stride=(2, 2), padding=1),
    nn.LeakyReLU(0.2, inplace=True),
    nn.Conv2d(64, 128, (3, 3), stride=(2, 2), padding=1),
    nn.LeakyReLU(0.2, inplace=True),
    GlobalMaxPool2d(),
    Flatten(),
    nn.Linear(128, 1),
)

model = {"generator": generator, "discriminator": discriminator}
criterion = {"generator": nn.BCEWithLogitsLoss(), "discriminator": nn.BCEWithLogitsLoss()}
optimizer = {
    "generator": torch.optim.Adam(generator.parameters(), lr=0.0003, betas=(0.5, 0.999)),
    "discriminator": torch.optim.Adam(discriminator.parameters(), lr=0.0003, betas=(0.5, 0.999)),
}
loaders = {"train": DataLoader(MNIST(os.getcwd(), train=False, download=True, transform=ToTensor()), batch_size=32)}

class CustomRunner(dl.Runner):
    def predict_batch(self, batch):
        batch_size = 1
        # Sample random points in the latent space
        random_latent_vectors = torch.randn(batch_size, latent_dim).to(self.device)
        # Decode them to fake images
        generated_images = self.model["generator"](random_latent_vectors).detach()
        return generated_images

    def handle_batch(self, batch):
        real_images, _ = batch
        batch_size = real_images.shape[0]

        # Sample random points in the latent space
        random_latent_vectors = torch.randn(batch_size, latent_dim).to(self.device)

        # Decode them to fake images
        generated_images = self.model["generator"](random_latent_vectors).detach()
        # Combine them with real images
        combined_images = torch.cat([generated_images, real_images])

        # Assemble labels discriminating real from fake images
        labels = torch.cat([torch.ones((batch_size, 1)), torch.zeros((batch_size, 1))]).to(self.device)
        # Add random noise to the labels - important trick!
        labels += 0.05 * torch.rand(labels.shape).to(self.device)

        # Discriminator forward
        combined_predictions = self.model["discriminator"](combined_images)

        # Sample random points in the latent space
        random_latent_vectors = torch.randn(batch_size, latent_dim).to(self.device)
        # Assemble labels that say "all real images"
        misleading_labels = torch.zeros((batch_size, 1)).to(self.device)

        # Generator forward
        generated_images = self.model["generator"](random_latent_vectors)
        generated_predictions = self.model["discriminator"](generated_images)

        self.batch = {
            "combined_predictions": combined_predictions,
            "labels": labels,
            "generated_predictions": generated_predictions,
            "misleading_labels": misleading_labels,
        }


runner = CustomRunner()
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    callbacks=[
        dl.CriterionCallback(
            input_key="combined_predictions",
            target_key="labels",
            metric_key="loss_discriminator",
            criterion_key="discriminator",
        ),
        dl.CriterionCallback(
            input_key="generated_predictions",
            target_key="misleading_labels",
            metric_key="loss_generator",
            criterion_key="generator",
        ),
        dl.OptimizerCallback(model_key="generator", optimizer_key="generator", metric_key="loss_generator"),
        dl.OptimizerCallback(model_key="discriminator", optimizer_key="discriminator", metric_key="loss_discriminator"),
    ],
    valid_loader="train",
    valid_metric="loss_generator",
    minimize_valid_metric=True,
    num_epochs=20,
    verbose=True,
    logdir="./logs/gan",
)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(runner.predict_batch(None)[0, 0].cpu().numpy())

### Mnist funetuting

[![Catalyst logo](https://raw.githubusercontent.com/catalyst-team/catalyst-pics/master/third_party_pics/train-loop.png)](https://github.com/catalyst-team/catalyst#minimal-examples)

In [ ]:
import os
from torch import nn, optim
from torch.utils.data import DataLoader
from catalyst import dl, utils
from catalyst.contrib.datasets import MNIST
from catalyst.data.transforms import ToTensor


class CustomRunner(dl.IRunner):
    def __init__(self, logdir, device):
        super().__init__()
        self._logdir = logdir
        self._device = device

    def get_engine(self):
        return dl.DeviceEngine(self._device)

    def get_loggers(self):
        return {
            "console": dl.ConsoleLogger(),
            "csv": dl.CSVLogger(logdir=self._logdir),
            "tensorboard": dl.TensorboardLogger(logdir=self._logdir),
        }

    @property
    def stages(self):
        return ["train_freezed", "train_unfreezed"]

    def get_stage_len(self, stage: str) -> int:
        return 3

    def get_loaders(self, stage: str, epoch: int = None):
        loaders = {
            "train": DataLoader(MNIST(os.getcwd(), train=True, download=True, transform=ToTensor()), batch_size=32),
            "valid": DataLoader(MNIST(os.getcwd(), train=False, download=True, transform=ToTensor()), batch_size=32),
        }
        return loaders

    def get_model(self, stage: str):
        model = (
            self.model
            if self.model is not None
            else nn.Sequential(nn.Flatten(), nn.Linear(784, 128), nn.ReLU(), nn.Linear(128, 10))
        )
        if stage == "train_freezed":
            # freeze layer
            utils.set_requires_grad(model[1], False)
        else:
            utils.set_requires_grad(model, True)
        return model

    def get_criterion(self, stage: str):
        return nn.CrossEntropyLoss()

    def get_optimizer(self, stage: str, model):
        if stage == "train_freezed":
            return optim.Adam(model.parameters(), lr=1e-3)
        else:
            return optim.SGD(model.parameters(), lr=1e-1)

    def get_scheduler(self, stage: str, optimizer):
        return None

    def get_callbacks(self, stage: str):
        return {
            "criterion": dl.CriterionCallback(
                metric_key="loss", input_key="logits", target_key="targets"
            ),
            "optimizer": dl.OptimizerCallback(metric_key="loss"),
            # "scheduler": dl.SchedulerCallback(loader_key="valid", metric_key="loss"),
            # "accuracy": dl.AccuracyCallback(input_key="logits", target_key="targets", topk_args=(1, 3, 5)),
            # "classification": dl.PrecisionRecallF1SupportCallback(input_key="logits", target_key="targets", num_classes=10),
            # "confusion_matrix": dl.ConfusionMatrixCallback(input_key="logits", target_key="targets", num_classes=10),
            "checkpoint": dl.CheckpointCallback(self._logdir, loader_key="valid", metric_key="loss", minimize=True, save_n_best=3),
        }

    def handle_batch(self, batch):
        x, y = batch
        logits = self.model(x)

        self.batch = {
            "features": x,
            "targets": y,
            "logits": logits,
        }

runner = CustomRunner("./logs/finetuning", "cuda")
runner.run()

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs